# Battle of the Cities: Getting Venues from Foursquare API

In [1]:
import requests # library to handle requests
import pandas as pd 
import numpy as np
import json

### Bring in data

In [8]:
sf_hoods = pd.read_csv('../data/sf-locations.csv',delimiter=",")
nyc_hoods = pd.read_csv('../data/nyc-locations.csv')

sf_hoods.head()

,Neighborhood,Latitude,Longitude
0,Anza Vista,37.7808,-122.4432
1,Alamo Square,37.7764,-122.4346
2,Sutro Heights,37.7782,-122.5083
3,Seacliff,37.7868,-122.4890
4,Lake Street,37.7856,-122.4794


## Foursquare requests

In [3]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IW1DNBRJOUHECGXR0ST002ZPDJYOBCIRTROZZ5YJZFHCIP3Q
CLIENT_SECRET:TWMIYINYFXOLI0IGNPXXLCIRHFWLUNJ4VP2TJZXSJGUP2ZBN


### Generate url for request

In [9]:
LIMIT = 50    
def getNearbyVenues(names, latitudes, longitudes, radius=300):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Turn `nearby_venues` into its own dataframe
#### San Francisco

In [10]:
sf_venues = getNearbyVenues(names=sf_hoods['Neighborhood'],
                                   latitudes=sf_hoods['Latitude'],
                                   longitudes=sf_hoods['Longitude']
                                  )

print(sf_venues.shape)
sf_venues.head()

Anza Vista
Alamo Square
Sutro Heights
Seacliff
Lake Street
Presidio
Presidio Terrace
Outer Richmond
Outer Sunset
Lakeshore
Inner Richmond
Inner Sunset
Fishermans Wharf
Aquatic Park / Ft. Mason
Union Street
Presidio Heights
Laurel Heights / Jordan Park
Northern Waterfront
Lone Mountain
Panhandle
Haight Ashbury
Ashbury Heights
Castro
Noe Valley
Glen Park
Outer Mission
Marina
Cow Hollow
Pacific Heights
Lower Pacific Heights
Japantown
Polk Gulch
Rincon Hill
Western Addition
Lower Haight
Duboce Triangle
Mint Hill
Mission Dolores
Hayes Valley
North Beach
Russian Hill
Chinatown
Nob Hill
Lower Nob Hill
Cathedral Hill
Downtown
Tenderloin
Civic Center
South of Market
Mission
Bernal Heights
Telegraph Hill
Financial District
South Beach
Showplace Square
Protrero Hill
Mission Bay
Dogpatch
Bayview
Central Waterfront
Golden Gate Heights
Buena Vista
Corona Heights
Cole Valley
Parnassus Heights
Eureka Valley
Dolores Heights
Forest Knolls
Clarendon Heights
Upper Market
Midtown Terrace
Laguna Honda
Fores

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Anza Vista,37.7808,-122.4432,Anza Vista,37.779721,-122.442065,Neighborhood
1,Anza Vista,37.7808,-122.4432,European Wax Center San Francisco,37.781791,-122.445398,Health & Beauty Service
2,Anza Vista,37.7808,-122.4432,European Wax Center,37.781838,-122.445449,Health & Beauty Service
3,Anza Vista,37.7808,-122.4432,Tony's Cable Car Restaurant,37.782814,-122.444836,Burger Joint
4,Anza Vista,37.7808,-122.4432,Subway,37.781817,-122.445434,Sandwich Place


#### New York City

In [12]:
nyc_venues = getNearbyVenues(names=nyc_hoods['Neighborhood'],
                                   latitudes=nyc_hoods['Latitude'],
                                   longitudes=nyc_hoods['Longitude']
                                  )

print(nyc_venues.shape)
nyc_venues.head()

Marble Hill
Inwood
Washington Heights
Hamilton Heights
Manhattanville
Morningside Heights
Harlem
East Harlem
Upper West Side
Carnegie Hill
Upper East Side
Roosevelt Island
Columbus Circle
Midtown
Clinton
Sutton Place
Turtle Bay
Tudor City
Garment District
Murry Hill
Chelsea
Flatiron District
Gramercy
Stuyvesant Town
West Village
Greenwich Village
East Village
SoHo
NoHo
Little Italy
Lower East Side
Tribeca
Battery Park
Financial District
Chinatown
(1371, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.8758,-73.9105,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.8758,-73.9105,Starbucks,40.873755,-73.908613,Coffee Shop
2,Marble Hill,40.8758,-73.9105,Rite Aid,40.875467,-73.908906,Pharmacy
3,Marble Hill,40.8758,-73.9105,Subway Sandwiches,40.874667,-73.909586,Sandwich Place
4,Marble Hill,40.8758,-73.9105,GameStop,40.874267,-73.909342,Video Game Store


### Group the venues by neighborhood

In [11]:
sf_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Alamo Square,17,17,17,17,17,17
Anza Vista,5,5,5,5,5,5
Apparel City,12,12,12,12,12,12
Aquatic Park / Ft. Mason,29,29,29,29,29,29
Ashbury Heights,9,9,9,9,9,9
...,...,...,...,...,...,...
West Portal,41,41,41,41,41,41
Western Addition,19,19,19,19,19,19
Westwood Highlands,1,1,1,1,1,1


In [13]:
nyc_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park,50,50,50,50,50,50
Carnegie Hill,25,25,25,25,25,25
Chelsea,50,50,50,50,50,50
Chinatown,50,50,50,50,50,50
Clinton,50,50,50,50,50,50
Columbus Circle,40,40,40,40,40,40
East Harlem,35,35,35,35,35,35
East Village,50,50,50,50,50,50
Financial District,50,50,50,50,50,50


## One Hot Encoding
### San Francisco

In [15]:
# one hot encoding
sf_onehot = pd.get_dummies(sf_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
sf_onehot['Neighborhood'] = sf_venues['Neighborhood'] 

sf_onehot.head()

,ATM,Acai House,Accessories Store,Adult Boutique,African Restaurant,Alternative Healer,American Restaurant,Antique Shop,Arcade,Art Gallery,...,Vietnamese Restaurant,Vineyard,Warehouse,Warehouse Store,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
# move neighborhood column to the first column
cols = sf_onehot.columns.to_list()
cols.insert(0, cols.pop(cols.index('Neighborhood')))
sf_onehot = sf_onehot.reindex(columns = cols)
sf_onehot.columns

Index(['Neighborhood', 'ATM', 'Acai House', 'Accessories Store',
       'Adult Boutique', 'African Restaurant', 'Alternative Healer',
       'American Restaurant', 'Antique Shop', 'Arcade',
       ...
       'Vietnamese Restaurant', 'Vineyard', 'Warehouse', 'Warehouse Store',
       'Wine Bar', 'Wine Shop', 'Winery', 'Wings Joint', 'Women's Store',
       'Yoga Studio'],
      dtype='object', length=317)

In [22]:
sf_onehot.head()

,Neighborhood,ATM,Acai House,Accessories Store,Adult Boutique,African Restaurant,Alternative Healer,American Restaurant,Antique Shop,Arcade,...,Vietnamese Restaurant,Vineyard,Warehouse,Warehouse Store,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,Anza Vista,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Anza Vista,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Anza Vista,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Anza Vista,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Anza Vista,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Now that I have the desired dataframe, I'll group the data by neighborhood and calculate the mean of the frequency of occurence of each category.

In [24]:
sf_grouped = sf_onehot.groupby('Neighborhood').mean().reset_index()
sf_grouped.head()

,Neighborhood,ATM,Acai House,Accessories Store,Adult Boutique,African Restaurant,Alternative Healer,American Restaurant,Antique Shop,Arcade,...,Vietnamese Restaurant,Vineyard,Warehouse,Warehouse Store,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,Alamo Square,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.058824,0.058824,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
1,Anza Vista,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
2,Apparel City,0.0,0.0,0.0,0.0,0.0,0.0,0.083333,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
3,Aquatic Park / Ft. Mason,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.034483,0.0,0.0,0.0
4,Ashbury Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.111111,0.0,0.000000,0.0,0.0,0.0


### New York City

In [25]:
# one hot encoding
nyc_onehot = pd.get_dummies(nyc_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
nyc_onehot['Neighborhood'] = nyc_venues['Neighborhood'] 

# move neighborhood column to the first column
cols = nyc_onehot.columns.to_list()
cols.insert(0, cols.pop(cols.index('Neighborhood')))
nyc_onehot = nyc_onehot.reindex(columns = cols)
nyc_onehot.columns

nyc_onehot.head()

,Neighborhood,Accessories Store,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [26]:
nyc_grouped = nyc_onehot.groupby('Neighborhood').mean().reset_index()
nyc_grouped.head()

,Neighborhood,Accessories Store,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Battery Park,0.0,0.0,0.02,0.0,0.0,0.00,0.00,0.0,0.0,...,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.00
1,Carnegie Hill,0.0,0.0,0.00,0.0,0.0,0.00,0.04,0.0,0.0,...,0.0,0.04,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.04
2,Chelsea,0.0,0.0,0.00,0.0,0.0,0.38,0.00,0.0,0.0,...,0.0,0.02,0.0,0.00,0.0,0.0,0.02,0.02,0.0,0.00
3,Chinatown,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,...,0.0,0.00,0.0,0.04,0.0,0.0,0.00,0.00,0.0,0.00
4,Clinton,0.0,0.0,0.04,0.0,0.0,0.00,0.00,0.0,0.0,...,0.0,0.00,0.0,0.00,0.0,0.0,0.02,0.06,0.0,0.00


## Get most common venues in each neighborhood

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### San Francisco

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
sf_hoods_venues_sorted = pd.DataFrame(columns=columns)
sf_hoods_venues_sorted['Neighborhood'] = sf_grouped['Neighborhood']

for ind in np.arange(sf_grouped.shape[0]):
    sf_hoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sf_grouped.iloc[ind, :], num_top_venues)

sf_hoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alamo Square,Park,Bus Station,Seafood Restaurant,Rock Club,Dog Run,Nightclub,Café,Historic Site,Liquor Store,Bakery
1,Anza Vista,Health & Beauty Service,Sandwich Place,Burger Joint,Yoga Studio,Farmers Market,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant
2,Apparel City,Nightclub,Miscellaneous Shop,Rental Car Location,Pet Service,Hardware Store,Automotive Shop,Garden Center,Outdoor Supply Store,Food Truck,Convenience Store
3,Aquatic Park / Ft. Mason,Chocolate Shop,Park,Gift Shop,Playground,Cantonese Restaurant,Shopping Plaza,Beer Garden,Garden,Mini Golf,Trail
4,Ashbury Heights,Italian Restaurant,Bakery,Sushi Restaurant,Organic Grocery,Gym,Bar,Breakfast Spot,Playground,Wine Bar,Department Store


### New York City

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
nyc_hoods_venues_sorted = pd.DataFrame(columns=columns)
nyc_hoods_venues_sorted['Neighborhood'] = nyc_grouped['Neighborhood']

for ind in np.arange(nyc_grouped.shape[0]):
    nyc_hoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(nyc_grouped.iloc[ind, :], num_top_venues)

nyc_hoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park,Park,Memorial Site,Coffee Shop,Cupcake Shop,Sandwich Place,Food Court,Shopping Mall,Boat or Ferry,Steakhouse,Gastropub
1,Carnegie Hill,Italian Restaurant,French Restaurant,Spa,Yoga Studio,Kosher Restaurant,Breakfast Spot,Sculpture Garden,Museum,Bookstore,Exhibit
2,Chelsea,Art Gallery,French Restaurant,Tapas Restaurant,Coffee Shop,Italian Restaurant,Park,Chinese Restaurant,Sandwich Place,Cupcake Shop,Bookstore
3,Chinatown,Chinese Restaurant,Bakery,Spa,Dessert Shop,Optical Shop,Dim Sum Restaurant,Cocktail Bar,Salon / Barbershop,Supermarket,Ice Cream Shop
4,Clinton,Theater,Wine Shop,New American Restaurant,American Restaurant,Bar,Lounge,Gym / Fitness Center,Hotel,Hotel Bar,Sporting Goods Shop
